 # Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this assignment, I will explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. I will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

Once the data is in a structured format, I will explore and cluster the neighborhoods in the city of Toronto.

Firstly, let's download all the packages that we will require to complete the analysis
1. Folium
2. Seaborn


In [6]:
!pip install folium

In [7]:
!pip install beautifulsoup4

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json
from bs4 import BeautifulSoup

%matplotlib inline

The neighborhood data for Toronto is available on [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

We will scrape this page using Beautifulsoup library and transform it into a dataset

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

resp = requests.get('https://en.wikipedia.org/wiki/Toronto')
soup = BeautifulSoup(resp.content)

In [5]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Wikimedia Error\n  </title>\n  <style>\n   * { margin: 0; padding: 0; }\nbody { background: #fff; font: 15px/1.6 sans-serif; color: #333; }\n.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }\n.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }\nimg { float: left; margin: 0 2em 2em 0; }\na img { border: 0; }\nh1 { margin-top: 1em; font-size: 1.2em; }\n.content-text { overflow: hidden; overflow-wrap: break-word; word-wrap: break-word; -webkit-hyphens: auto; -moz-hyphens: auto; -ms-hyphens: auto; hyphens: auto; }\np { margin: 0.7em 0 1em 0; }\na { color: #0645ad; text-decoration: none; }\na:hover { text-decoration: underline; }\ncode { font-family: sans-serif; }\n.text-muted { color: #777; }\n  </style>\n </head>\n <body>\n  <div class="content" role="main">\n   <a href="https://www.wikim

Since I'm getting an error from Wikipedia side, I'll download the html and then parse using beautiful soup

<b>PS:</b> One block of code is hidden since it contains client id and secret to pull HTML from IBM data storage

In [80]:
# The code was removed by Watson Studio for sharing.

In [81]:
soup = BeautifulSoup(toronto_html)

In [82]:
neighborhood_table = soup.find('table')

In [83]:
table_str = str(neighborhood_table.extract())

In [84]:
toronto_df = pd.read_html(table_str)[0]

In [85]:
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Pre-Processing

In [86]:
toronto_df.columns = ['PostalCode'] + list(toronto_df.columns)[1:]

In [87]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [89]:
toronto_df.shape

(287, 3)

I am removing the cells with a borough that is Not assigned

In [90]:
toronto_df.Borough.value_counts()

Not assigned        77
Etobicoke           44
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [91]:
borough_na = toronto_df[toronto_df.Borough == 'Not assigned']
toronto_df.drop(borough_na.index, inplace=True)

In [92]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [93]:
toronto_df.shape

(210, 3)

Next, I will combine neighborhoods with the same postal code into one row

In [106]:
toronto_df = toronto_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame()

In [107]:
toronto_df.reset_index(inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let's check whether all Neighorhoods are valid

In [113]:
toronto_df[toronto_df.Neighborhood == "Not assigned"]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [117]:
toronto_df.loc[85, 'Neighborhood'] = toronto_df.loc[85, 'Borough']
toronto_df[toronto_df.Neighborhood == "Not assigned"]

,PostalCode,Borough,Neighborhood


Following is the cleaned and processed dataframe

In [ ]:
toronto_df.head()